# 12 Lead ST-wave Analysis

In [1]:
import warnings
warnings.simplefilter("ignore")

import os
import pandas as pd
import wfdb
import ast
import numpy as np
import matplotlib.pyplot as plt
# ============== Scipy
from scipy import signal
from scipy.signal import butter, lfilter, freqz, filtfilt


# ============== Jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(threshold=np.inf)

In [2]:
ptb_xl_path = "/data/ECG_DATASET/ptbxl/"
os.listdir(ptb_xl_path)

['scp_statements.csv',
 'records500',
 'ptbxl_v102_changelog.txt',
 'records100',
 'SHA256SUMS.txt',
 'example_physionet.py',
 'RECORDS',
 'ptbxl_database.csv',
 'LICENSE.txt',
 'ptbxl_v103_changelog.txt']

### Example_Physionet.py

In [3]:
ptb_db = pd.read_csv(ptb_xl_path+"/ptbxl_database.csv")

In [4]:
def load_raw_data(df, sampling_rate, path):
    data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))


In [5]:
sampling_rate = 500
ptb_xl_path = "/data/ECG_DATASET/ptbxl/"

In [6]:
Y = pd.read_csv(ptb_xl_path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

In [7]:
X = load_raw_data(Y, sampling_rate, ptb_xl_path)

In [8]:
X.shape

(21799, 5000, 12)

In [9]:
X.shape

# 21799 clinical 12-lead ECG records of 10 seconds length from 18869 patients => 21799
# 10 sec / 500Hz => 5000
# Lead : 12

(21799, 5000, 12)

In [10]:
# # Plotting Example Data about 12-Lead with One Person

# %matplotlib INLINE

# fig = plt.figure(figsize=(40,10))
# for i,data in enumerate(test_D):
#     ax = fig.add_subplot(2,6,i+1)
#     ax.plot(data)
# plt.show()

In [11]:
Y.columns

Index(['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site',
       'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr'],
      dtype='object')

In [12]:
agg_df = pd.read_csv(ptb_xl_path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]
agg_df

description  diagnostic  form  \
NDT                         non-diagnostic T abnormalities         1.0   1.0   
NST_                               non-specific ST changes         1.0   1.0   
DIG                                       digitalis-effect         1.0   1.0   
LNGQT                                     long QT-interval         1.0   1.0   
NORM                                            normal ECG         1.0   NaN   
IMI                         inferior myocardial infarction         1.0   NaN   
ASMI                    anteroseptal myocardial infarction         1.0   NaN   
LVH                           left ventricular hypertrophy         1.0   NaN   
LAFB                        left anterior fascicular block         1.0   NaN   
ISC_                                 non-specific ischemic         1.0   NaN   
IRBBB                 incomplete right bundle branch block         1.0   NaN   
1AVB                                 first degree AV block         1.0   NaN   
IVCD     non-specific intraventricular conduction distu...         1.0   NaN   
ISCAL                      ischemic in anterolateral leads         1.0   NaN   
CRBBB                   complete right bundle branch block         1.0   NaN   
CLBBB                    complete left bundle branch block         1.0   NaN   
ILMI                   inferolateral myocardial infarction         1.0   NaN   
LAO/LAE                   left atrial overload/enlargement         1.0   NaN   
AMI                         anterior myocardial infarction         1.0   NaN   
ALMI                   anterolateral myocardial infarction         1.0   NaN   
ISCIN                           ischemic in inferior leads         1.0   NaN   
INJAS          subendocardial injury in anteroseptal leads         1.0   NaN   
LMI                          lateral myocardial infarction         1.0   NaN   
ISCIL                      ischemic in inferolateral leads         1.0   NaN   
LPFB                       left posterior fascicular block         1.0   NaN   
ISCAS                       ischemic in anteroseptal leads         1.0   NaN   
INJAL         subendocardial injury in anterolateral leads         1.0   NaN   
ISCLA                            ischemic in lateral leads         1.0   NaN   
RVH                          right ventricular hypertrophy         1.0   NaN   
ANEUR    ST-T changes compatible with ventricular aneurysm         1.0   NaN   
RAO/RAE                  right atrial overload/enlargement         1.0   NaN   
EL          electrolytic disturbance or drug (former EDIS)         1.0   NaN   
WPW                          Wolf-Parkinson-White syndrome         1.0   NaN   
ILBBB                  incomplete left bundle branch block         1.0   NaN   
IPLMI           inferoposterolateral myocardial infarction         1.0   NaN   
ISCAN                           ischemic in anterior leads         1.0   NaN   
IPMI                 inferoposterior myocardial infarction         1.0   NaN   
SEHYP                                   septal hypertrophy         1.0   NaN   
INJIN              subendocardial injury in inferior leads         1.0   NaN   
INJLA               subendocardial injury in lateral leads         1.0   NaN   
PMI                        posterior myocardial infarction         1.0   NaN   
3AVB                                 third degree AV block         1.0   NaN   
INJIL         subendocardial injury in inferolateral leads         1.0   NaN   
2AVB                                second degree AV block         1.0   NaN   

         rhythm diagnostic_class diagnostic_subclass  \
NDT         NaN             STTC                STTC   
NST_        NaN             STTC                NST_   
DIG         NaN             STTC                STTC   
LNGQT       NaN             STTC                STTC   
NORM        NaN             NORM                NORM   
IMI         NaN               MI                 IMI   
ASMI        NaN               MI                 AMI  

In [13]:
# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Split data into train and test
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass

In [14]:
for y in y_train:
    print(y)

['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['MI']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
[]
[]
['NORM']
[]
['NORM']
['STTC']
[]
['NORM']
['NORM']
['STTC']
['NORM']
['STTC']
['NORM']
['HYP']
['NORM']
['CD']
['NORM']
[]
['NORM']
['NORM']
['NORM']
['MI', 'STTC']
['CD']
['NORM']
['NORM']
['NORM']
['HYP', 'CD']
['NORM']
['NORM']
['STTC']
['CD']
['MI', 'CD']
['NORM']
['CD']
['NORM']
['STTC']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['MI', 'CD']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['STTC', 'CD']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['HYP']
['NORM']
['NORM']
['NORM']
['NORM']
['STTC', 'CD']
['MI', 'CD']
['NORM']
['NORM']
['MI', 'HYP']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['NORM']
['STTC']
['NORM']
['MI']
['STTC']
['STT

In [15]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(19601, 5000, 12) (19601,)
(2198, 5000, 12) (2198,)


In [16]:
print(y_train)

ecg_id
1        [NORM]
2        [NORM]
3        [NORM]
4        [NORM]
5        [NORM]
          ...  
21833    [STTC]
21834    [NORM]
21835    [STTC]
21836    [NORM]
21837    [NORM]
Name: diagnostic_superclass, Length: 19601, dtype: object


Header에 따르면 순서는 다음과 같다.
* I, II, III, AVR, AVL, AVF, V1, V2, V3, V4, V5, V6
총 12개

In [17]:
print(compare_data2[: , 0]) # I번째 리드의 10초 데이터

NameError: name 'compare_data2' is not defined

In [ ]:
# Plotting Example Data about 12-Lead with One Person
%matplotlib INLINE

fig = plt.figure(figsize=(40,10))
ax = fig.add_subplot(1,1,1)
ax.plot(compare_data2[: , 0])
plt.show()

### Data Preprocessing (Noise?)

1. Baseline
2. Noise Filtering

 > Standardized Database of 12-Lead  Electrocardiograms with a Common Standard for the Promotion of Cardiovascular Research: KURIAS-ECG  
 
 In this study, the cut-off frequency was set from **0.5 to 150 Hz** to minimize the distortion of the ST segment and to maintain the post-potential information of the QRS wave

In [ ]:
test_data = compare_data2[: , 0]

def lowpass_filt(fs, low, order=5):
    nyq = 0.5 * fs # 250Hz
#     lowcut = ((2*low)/fs)
    lowcut = low / nyq
    b, a = butter(order, lowcut, btype="low")
    return b, a

def bandpass_filt(fs, low, high, order=5):
    nyq = 0.5 * fs
    lowcut = low / nyq 
    highcut = high / nyq
    b,a = butter(order, [lowcut, highcut], btype="band")
    return b, a

b_l, a_l = lowpass_filt(500, 45, 15) # PowerLine Noise Removal
b_b, a_b = bandpass_filt(500, 0.5, 150, 6) # 
filtered_ecg_data = lfilter(b_l, a_l, test_data)

In [ ]:
%matplotlib INLINE

fig = plt.figure(figsize=(40,10))
ax1 = fig.add_subplot(2,1,1)
ax1.plot(test_data)

ax2 = fig.add_subplot(2,1,2)
ax2.plot(filtered_ecg_data)

plt.show()

#### Baseline Wander Remove